<a href="https://colab.research.google.com/github/cvalencia15/Algoritmos-de-Optimizacion/blob/main/Carolina_Valencia_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **ALGORITMOS DE OPTIMIZACIÓN - ACTIVIDAD GUIADA 3 (AG3)**

Carolina Valencia López

URL: https://colab.research.google.com/drive/1adbmUcHs6jV9I-MdIQ88ZUC9pvlFKsZr?usp=sharing

GitHub: 

Problema del agente viajero con TSPLIB

In [14]:
!pip install requests
!pip install tsplib95

import tsplib95
import random
from math import e
import copy
import urllib.request

file="swiss42.tsp"
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp",file)

#se carga el problema
problem=tsplib95.load_problem(file)
#se definen los nodos
nodos=list(problem.get_nodes())
#se definen las aristas
aristas=list(problem.get_edges())
#dist entre nodos
problem.get_weight(0,1)


#probamos con otro problema, como el de bridge tournament (he jugado mucho al bridge, y me interesa)
#torneo de bridge con "estrategia de planificacion" round robin
#dimension=180 , 
file2="brg180.tsp"
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/brg180.tsp",file2)

#se carga el problema
problem2=tsplib95.load_problem(file2)
#se definen los nodos
nodos2=list(problem.get_nodes())
#se definen las aristas
aristas2=list(problem.get_edges())
#dist entre nodos
problem2.get_weight(0,2)
#la mejor solucion alcanzada es 1950


#probamos con otro problema que contiene ciclo hamiltoniano
#en http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf se menciona que los problemas dotados de extension .opt.tour contienen ciclo hamiltoniano

file3="kroD100.opt.tour"
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/kroD100.opt.tour",file3)
#se carga el problema
problem3=tsplib95.load_problem(file3)
#se definen los nodos
nodos3=list(problem.get_nodes())
#se definen las aristas
aristas3=list(problem.get_edges())
#dist entre nodos
problem2.get_weight(0,1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated function (or staticmethod) load_problem. (Will be removed in newer versions. Use `tsplib95.load` instead.) -- Deprecated since version 7.0.0.


20

Resolucion por busqueda aleatoria

In [20]:
#algoritmo de busqueda aleatoria en el que se van generando
#soluciones aleatorias en cada iterac, retornando la mejor

def busqueda_aleatoria(problem,n):
  nodos=list(problem.get_nodes())

  mejor_sol=[]
  mejor_dist=10e100

  for i in range(n):
    sol=crear_solucion(nodos)
    dist=distancia_total(sol,problem)

    if (dist<mejor_dist):
      mejor_sol=sol
      mejor_dist=dist

  return mejor_sol


#funcion para generar solucion aleatoria empezando por nodo 0
def crear_solucion(nodos):
  sol=[nodos[0]]
  for n in nodos[1:]:
    sol=sol+[random.choice(list(set(nodos)-set({nodos[0]}) - set(sol)))]
  return sol


#funcion que retorna la dist entre dos nodos
def distancia(a,b,problem):
  return problem.get_weight(a,b)


#funcion que retorna la dist total de una trayectoria o soluc
def distancia_total(sol,problem):
  dtotal=0
  for i in range(len(sol)-1):
    dtotal+=distancia(sol[i], sol[i+1],problem)
  return dtotal+distancia(sol[len(sol)-1], sol[0], problem)

#invocamos a la funcion y test
busqueda_aleatoria(problem,5000)


    

[0,
 35,
 17,
 13,
 28,
 30,
 33,
 29,
 11,
 2,
 19,
 32,
 31,
 7,
 5,
 12,
 22,
 40,
 21,
 8,
 39,
 4,
 37,
 15,
 6,
 25,
 3,
 27,
 38,
 24,
 23,
 34,
 20,
 26,
 41,
 9,
 16,
 14,
 1,
 10,
 18,
 36]

Resolución por busqueda local

In [22]:
'''En primer lugar, voy a ofrecer una posible implementacion a modo de 
pseudocodigo del algoritmo Hill Climbing (o de escalada, en español), 
que es un metodo de optimizacion encuadrado dentro de 
los algoritmos de busqueda local.

Su funcionamiento es sencillo: se empieza buscando desde un punto.
Si el nuevo punto ofrece una mejor solucion: punto_nuevo=punto_actual;
en caso contrario, se elige otro punto neighbour y se analiza.
El fin del algoritmo viene dado bien porque el numero de iteraciones
ya se ha agotado, o porque no se ha encontrado un punto que mejore
al actual.

Es posible que no haya solucion por la aparicion de un maximo local, o
simplemente porque los neighbours tienen el mismo valor.

Lo voy a presentar como un problema TSP o del viajero,
que hemos visto anteriormente

def escalada(problem):
  #empezamos seleccionando una solucion aleatoria
  sol =soluc_aux_aleat(problem)
  trayectoria=trayect(problem,sol)
  vecinos=get_vecinos(problem)  #se toman los vecinos, probabl. a traves de otra funcion, como knn
  vecino_mas_prometedor=get_vecino_mas_prometedor(problem,vecinos)  #se toman los vecinos mas prometedores, ya que mejoran la sol actual
  trayect_vecino_mas_prometedor=get_vecino_mas_prometedor(problem,vecinos)

  mientras que la trayect_vecino_mas_prometedor < trayectoria:  #se va comparando la situac actual y la del vecino, para ver si se actualiza
    sol=vecino_mas_prometedor 
    trayectoria=trayect_vecino_mas_prometedor
    vecinos=get_vecinos(problem)
    vecino_mas_prometedor=get_vecino_mas_prometedor(problem,vecinos)
    trayect_vecino_mas_prometedor=get_vecino_mas_prometedor(problem,vecinos)

  retornar sol #se retorna la solucion

'''

#Ahora, el algoritmo de BL visto en clase

def busqueda_local(problem):
  mejor_sol=[]

  #genera solucion aleatoria
  sol_referencia=crear_solucion(nodos)
  mejor_dist=distancia_total(sol_referencia,problem)

  iteracion=0
  while(1):
    iteracion+=1 #incr contador

    #obtener vecinos
    vecina=genera_vecina(sol_referencia)
    dist_vecina=distancia_total(vecina,problem)

    #se ha llegado a min local - si no mejoria, fin
    if (dist_vecina<mejor_dist):
      mejor_sol=copy.deepcopy(vecina)
      mejor_dist=dist_vecina
    else:
      print("En la iteracion",iteracion,"la mejor solucion es",mejor_sol)
      return mejor_sol


#funcion generadora de soluciones vecinas
#con n nodos -> (n-1)*(n-1)/2 sols
def genera_vecina(sol):
  mejor_sol=[]
  mejor_dist=10e100
  #recorrido del array de sols (nodo) en bucle doble para evaluarlos 2 a 2
  for i in range(1,len(sol)-1):
    for j in range(i+1,len(sol)):
      vecina=sol[:i]+[sol[j]] + sol[i+1:j] + [sol[i]] + sol[j+1:]
      
      #evaluacion de la solucion
      dist_vecina=distancia_total(vecina,problem)

      #se almacena en caso de ser mas optima
      if (dist_vecina<=mejor_dist):
        mejor_dist=dist_vecina
        mejor_sol=vecina
    return mejor_sol

#invocamos a la funcion y test
busqueda_local(problem)



En la iteracion 2 la mejor solucion es [0, 24, 21, 38, 33, 36, 8, 40, 11, 4, 34, 30, 32, 18, 9, 2, 3, 22, 5, 12, 1, 23, 13, 41, 37, 35, 29, 16, 39, 25, 19, 27, 6, 17, 15, 31, 14, 7, 20, 26, 10, 28]


[0,
 24,
 21,
 38,
 33,
 36,
 8,
 40,
 11,
 4,
 34,
 30,
 32,
 18,
 9,
 2,
 3,
 22,
 5,
 12,
 1,
 23,
 13,
 41,
 37,
 35,
 29,
 16,
 39,
 25,
 19,
 27,
 6,
 17,
 15,
 31,
 14,
 7,
 20,
 26,
 10,
 28]

Resolución por recocido simulado

In [29]:
#Funcion de recocido simulado (para problemas de optimiz global, no local)
#Es una func de probabilidad p(n) que depende de T y de la dif de costes
import math

def recocido_simulado(problem,t):
  sol_ref=crear_solucion(nodos)
  dist_ref=distancia_total(sol_ref,problem)

  mejor_sol=[]
  mejor_dist=10e100
  n=0

  while t>0.0001:
    n+=1
    #generacion de sol vecina
    vecina=genera_vecina_aleat(sol_ref)

    #calcula el valor
    dist_vecina=distancia_total(vecina,problem)

    #se almacena la sol
    if (dist_vecina<mejor_dist):
      mejor_sol=vecina
      mejor_dist=dist_vecina

    #si el nuevo vecino es mejor->update; si no, se evalua segun t y delta(dist_ref-dist_vecino)
    if (dist_vecina<dist_ref or probabilidad(t,abs(dist_ref-dist_vecina))):
      sol_ref=copy.deepcopy(vecina)
      dist_ref=dist_vecina

    #decrementar temperatura
    t=bajar_temperatura(t)

  print("La mejor solucion es", mejor_sol,"con un distancia total de", mejor_dist)
  return mejor_sol


#funcion de probab para evaluar si se debe cambiar a otra sol peor que la de ref
def probabilidad(t,d):
  if (random.random()<=math.exp(-1*d/t)):
    return True
  else:
    return False

#funcion para bajar la temperatura
def bajar_temperatura(t):
  return t*0.999

#funcion que genera una solucion vecina 2opt aleat
def genera_vecina_aleat(sol):
  #se seleccionan 2 nodos aleat
  i,j=sorted(random.sample(range(1,len(sol)),2))

  #retorna una sol nueva pero intercambiando los nodos selecc aleat
  return sol[:i]+[sol[j]]+sol[i+1:j]+[sol[i]]+sol[j+1:]


#invocacion a la funcion y test
recocido_simulado(problem,10000000)

La mejor solucion es [0, 1, 6, 5, 26, 18, 13, 19, 14, 16, 15, 37, 36, 35, 31, 17, 7, 32, 28, 2, 27, 3, 4, 12, 11, 25, 10, 8, 9, 39, 21, 24, 40, 23, 41, 22, 38, 29, 30, 34, 33, 20] con un distancia total de 1530


[0,
 1,
 6,
 5,
 26,
 18,
 13,
 19,
 14,
 16,
 15,
 37,
 36,
 35,
 31,
 17,
 7,
 32,
 28,
 2,
 27,
 3,
 4,
 12,
 11,
 25,
 10,
 8,
 9,
 39,
 21,
 24,
 40,
 23,
 41,
 22,
 38,
 29,
 30,
 34,
 33,
 20]

TAREAS OPCIONALES
(en las anteriores AG las he anadido a modo de comentario, pero aquí las pondre en esta celda)



*   **Búsqueda local con Entornos variables**
Se puede mejorar con la aplicación de una Búsqueda Descendente Basada en Entornos Variables. En este metodo, el operador de vecindad cambia de entorno, ampliando el espacio de busqueda para que no se caiga en mínimos locales (en el caso de que se proceda a un cambio de estructura de entornos al caer en minimos).


*   **Recocido simulado**
El recocido simulado puede mejorarse al no utilizar una técnica aleatoria de generación de vecindario. Sería una buena opción implementar una solucion de eleccion de vecinos mas compleja y que no caiga en la aleatoriedad para tambien, por ejemplo, evitar caer en minimos locales y que la busqueda en el espacio sea mas amplia.
La eficiencia del algoritmo tambien viene determinada por la estructura de colocacion de los vecinos, por lo que una tecnica mas optima que la meramente aleatoria tambien podria ayudar a mejorar la eficiencia.
Esto podria hacerse con un algoritmo/sistema basado en cadenas de Markov

